In [1]:
import pandas as pd
import numpy as np
import copy
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.tokenize import WordPunctTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import word_tokenize
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read datasets
dataframe = pd.read_csv("./data/RestoInfo.csv")

In [3]:
# Head of the data
dataframe.head()

,Unnamed: 0,name,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,46019,Unique Brew Cafe Resto,No,No,NaN,0,Indiranagar,Quick Bites,NaN,Fast Food,200,[],[],Dine-out,Old Airport Road
1,28849,Jayanthi Sagar,No,No,3.1 /5,21,Koramangala 5th Block,Quick Bites,NaN,"South Indian, North Indian, Chinese",200,"[('Rated 2.0', ""RATEDn Works only because it'...",[],Dine-out,Koramangala 4th Block
2,19855,Rock Stone Ice Cream Factory,Yes,No,4.0/5,131,BTM,Dessert Parlor,"Icecream Cake, Brownie, Waffles, Chocolate Ice...",Ice Cream,230,"[('Rated 4.0', ""RATEDn Ice creams are really ...","['Midnight Indulgence Cake', 'Butterscotch Mel...",Delivery,Jayanagar
3,35188,Punjabi by Nature 2.0,No,No,4.2 /5,3236,BTM,"Casual Dining, Microbrewery","Paneer Tikki, Mutton Raan, Mango Margarita, Cr...",North Indian,NaN,"[('Rated 3.0', ""RATEDn It has a beautiful amb...",[],Delivery,Koramangala 7th Block
4,7070,Rayalaseema Chefs,Yes,Yes,3.9/5,225,Marathahalli,Casual Dining,"Bamboo Chicken, Butter Naan, Mutton Biryani, P...","North Indian, Biryani, Andhra, Chinese",800,"[('Rated 5.0', 'RATEDn Had Good experience wi...",[],Delivery,Brookefield


In [4]:
# Tail of the data
dataframe.tail()

,Unnamed: 0,name,online_order,book_table,rate,votes,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
2064,8778,Zeeshan,Yes,No,3.8/5,214,Koramangala 7th Block,Casual Dining,"Rolls, Kathi Roll, Mutton Roll, Chicken Roll, ...","North Indian, Kebab, Biryani, Rolls",700,"[('Rated 1.0', ""RATEDn Pathetic service .we v...",[],Delivery,BTM
2065,18086,Zaica Dine & Wine,Yes,Yes,4.0/5,1013,Indiranagar,Casual Dining,"Chocolate Pani Puri, Butter Chicken, Gajar Hal...",North Indian,"1,200","[('Rated 5.0', ""RATEDn This place is located ...",[],Delivery,Indiranagar
2066,46112,Murphy's Brewhouse - The Paul Bangalore,No,Yes,4.3 /5,2039,Domlur,"Pub, Microbrewery","Chicken Bruschetta, Tandoori Chicken, Veggie P...","Continental, North Indian, Finger Food","1,700","[('Rated 4.0', 'RATEDn Service - 4/5, Taste -...",[],Pubs and bars,Old Airport Road
2067,10808,Cee Yem,No,No,NaN,0,BTM,Quick Bites,NaN,"South Indian, Kerala, North Indian",300,[],[],Dine-out,BTM
2068,28875,Cauvery Fast Food,No,No,3.2 /5,4,Ejipura,Quick Bites,NaN,Chinese,150,"[('Rated 3.0', 'RATEDn Small place with minim...",[],Dine-out,Koramangala 4th Block


In [5]:
# Resetting the index
dataframe.set_index('Unnamed: 0', inplace=True)

In [6]:
# EDA
# What are the data types?
dataframe.dtypes

name                           object
online_order                   object
book_table                     object
rate                           object
votes                           int64
location                       object
rest_type                      object
dish_liked                     object
cuisines                       object
approx_cost(for two people)    object
reviews_list                   object
menu_item                      object
listed_in(type)                object
listed_in(city)                object
dtype: object

In [7]:
# What is the distribution of numerical columns?
dataframe.describe()

,votes
count,2069.000000
mean,276.048333
std,673.916908
min,0.000000
25%,7.000000
50%,43.000000
75%,210.000000
max,7854.000000


In [8]:
# What about categorical variable levels count?
dataframe.describe(include=['object'])

,name,online_order,book_table,rate,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
count,2069,2069,2069,1770,2069,2052,962,2069,2053,2069,2069,2069,2069
unique,1665,2,2,55,82,64,825,771,46,1698,483,7,30
top,Lassi Shop,Yes,No,NEW,BTM,Quick Bites,Biryani,North Indian,300,[],[],Delivery,BTM
freq,6,1221,1810,89,227,796,7,130,299,268,1580,1048,154


## Check if there are any na values.

In [9]:
dataframe.isna().sum()

name                              0
online_order                      0
book_table                        0
rate                            299
votes                             0
location                          0
rest_type                        17
dish_liked                     1107
cuisines                          0
approx_cost(for two people)      16
reviews_list                      0
menu_item                         0
listed_in(type)                   0
listed_in(city)                   0
dtype: int64

# Imputing NA values

In [10]:
# Fill na values.
# Since the columns containing na values is of categorical type so filling the na vlaues with mode.
dataframe = dataframe.fillna(dataframe.mode().iloc[0])
dataframe.isna().sum()

name                           0
online_order                   0
book_table                     0
rate                           0
votes                          0
location                       0
rest_type                      0
dish_liked                     0
cuisines                       0
approx_cost(for two people)    0
reviews_list                   0
menu_item                      0
listed_in(type)                0
listed_in(city)                0
dtype: int64

In [11]:
# Checking the unique valies in all the column:
for i in dataframe.columns:
    print("column Name {} and unique_values\n\n {}".format(i, dataframe[i].unique()))
    print ("---------------------- \n")
    print("column Name {} and total number of unique_values\n {}\n\n".format(i, len(dataframe[i].unique())))

column Name name and unique_values

 ['Unique Brew Cafe Resto' 'Jayanthi Sagar' 'Rock Stone Ice Cream Factory'
 ... 'Zaica Dine & Wine' "Murphy's Brewhouse - The Paul Bangalore"
 'Cee Yem']
---------------------- 

column Name name and total number of unique_values
 1665


column Name online_order and unique_values

 ['No' 'Yes']
---------------------- 

column Name online_order and total number of unique_values
 2


column Name book_table and unique_values

 ['No' 'Yes']
---------------------- 

column Name book_table and total number of unique_values
 2


column Name rate and unique_values

 ['NEW' '3.1 /5' '4.0/5' '4.2 /5' '3.9/5' '4.1 /5' '3.3/5' '4.4/5' '3.8/5'
 '3.1/5' '3.5/5' '3.8 /5' '2.9/5' '4.1/5' '3.7/5' '3.5 /5' '3.4 /5'
 '3.7 /5' '4.5/5' '4.3 /5' '3.6/5' '2.8 /5' '3.2/5' '4.0 /5' '2.9 /5'
 '3.0 /5' '4.4 /5' '3.2 /5' '4.6 /5' '3.9 /5' '3.0/5' '4.8 /5' '3.6 /5'
 '3.3 /5' '4.5 /5' '2.8/5' '3.4/5' '4.2/5' '2.1/5' '4.9 /5' '4.3/5'
 '2.7/5' '2.6/5' '2.7 /5' '2.2/5' '2.4/5' '2.5 

column Name menu_item and unique_values

 ['[]'
 "['Midnight Indulgence Cake', 'Butterscotch Melody Cake', 'Rock With Ferrero Rocher Ice Cream', 'Our Beloved Classic Ice Cream', 'Vanilla French Kisssss Ice Cream', 'An Italian Love Affair Ice Cream', 'Oreo Double Double Ice Cream', 'Hot Chocolate No Mercy Ice Cream [Regular]', 'Lava Meltdown Ice Cream [Regular]', 'All Chocolate No Nonsence', 'Beyond Darkness Beyond Desire', 'Chocolate Passion for Strawberry', 'Nutella Frenzy', 'Doctor Nuts', 'Rocky Off Road', 'Exotic Fruit Trill', 'Fruit Extravaganza', 'Hot Chocolate No Mercy Ice Cream [Regular]', 'Lava Meltdown Ice Cream [Regular]', 'All Chocolate No Nonsence', 'Beyond Darkness Beyond Desire', 'Chocolate Passion for Strawberry', 'Nutella Frenzy', 'Doctor Nuts', 'Rocky Off Road', 'Exotic Fruit Trill', 'Fruit Extravaganza', 'Strawberry Razzmatazz', 'Litchi Double Double', 'Cookies N Creamery Cake', 'Midnight Indulgence Cake', 'Butterscotch Melody Cake', 'Caramel Nuts Paradise Cake', 'Rai

## Changing the columns to appropriate type.

In [12]:
# 1) changing the column "approx_cost(for two people)" numeric.


def remove_comma_symbol(df, comma_columns):
        """
        Reads a dataframe and removes the ',' symbol from the data



        returns the dataframe without comma symbol.
        """
        for col in comma_columns:
            df[col]=df[col].str.replace(',', '').astype('float')

        return df
        
dataframe = remove_comma_symbol(dataframe, ['approx_cost(for two people)'])

In [13]:
# 2) Keeping the values from rate column. Since we know all the ratings are upon 5 Ex: 3/5. So in the keeping only 3.

print(dataframe.rate.unique())
print(dataframe.rate.describe())
def remove_rating_denominator(df, rating_columns):
        """
        Reads a dataframe and removes the ',' symbol from the data



        returns the dataframe without comma symbol.
        """
        for col in rating_columns:
            df[col]=df[col].str.replace('NEW', '3.0') # Replacing 3 based on the 25% of the rate excluding new or -
            df[col]=df[col].str.replace('-', '3.0') # Replacing 3 based on the 25% of the rate excluding new or -
            df[col]=df[col].str.replace(' /5', '')
            df[col]=df[col].str.replace('/5', '')
            df[col]=df[col].astype('float')

        return df
dataframe = remove_rating_denominator(dataframe, ['rate'])

['NEW' '3.1 /5' '4.0/5' '4.2 /5' '3.9/5' '4.1 /5' '3.3/5' '4.4/5' '3.8/5'
 '3.1/5' '3.5/5' '3.8 /5' '2.9/5' '4.1/5' '3.7/5' '3.5 /5' '3.4 /5'
 '3.7 /5' '4.5/5' '4.3 /5' '3.6/5' '2.8 /5' '3.2/5' '4.0 /5' '2.9 /5'
 '3.0 /5' '4.4 /5' '3.2 /5' '4.6 /5' '3.9 /5' '3.0/5' '4.8 /5' '3.6 /5'
 '3.3 /5' '4.5 /5' '2.8/5' '3.4/5' '4.2/5' '2.1/5' '4.9 /5' '4.3/5'
 '2.7/5' '2.6/5' '2.7 /5' '2.2/5' '2.4/5' '2.5 /5' '2.1 /5' '1.8 /5'
 '2.6 /5' '4.7 /5' '-' '2.3/5' '4.6/5' '2.2 /5']
count     2069
unique      55
top        NEW
freq       388
Name: rate, dtype: object


In [14]:
# Seperating numeric, categorical, small text, large text and review columns.
num_attr = ['rate', 'votes', 'approx_cost(for two people)']
cat_attr = ['online_order', 'book_table', 'listed_in(type)']
small_text_attr = ['name', 'location', 'rest_type', 'listed_in(city)']
large_text_attr = ['dish_liked', 'cuisines', 'menu_item']
review = ['reviews_list']

In [15]:
# What is the distribution of numerical columns?
dataframe.describe()

,rate,votes,approx_cost(for two people)
count,2069.000000,2069.000000,2069.000000
mean,3.559111,276.048333,561.599807
std,0.485207,673.916908,468.672559
min,1.800000,0.000000,50.000000
25%,3.000000,7.000000,300.000000
50%,3.600000,43.000000,400.000000
75%,3.900000,210.000000,650.000000
max,4.900000,7854.000000,4000.000000


In [16]:
# # What about categorical variable levels count?
dataframe.describe(include=['object'])

,name,online_order,book_table,location,rest_type,dish_liked,cuisines,reviews_list,menu_item,listed_in(type),listed_in(city)
count,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069,2069
unique,1665,2,2,82,64,825,771,1698,483,7,30
top,Lassi Shop,Yes,No,BTM,Quick Bites,Biryani,North Indian,[],[],Delivery,BTM
freq,6,1221,1810,227,813,1114,130,268,1580,1048,154


In [17]:
# Model 1 Simple method of weighted rating based on each restaurant
print("\n\n Model 1\n\n")
top5_location = dataframe.location.value_counts().head()


#Number of restaurants in NCR with aggregate rating ranging from 1.9 to 4.9
top5_location = ['BTM','HSR','JP Nagar','Koramangala 5th Block', 'Marathahalli']
new_df = dataframe[(dataframe.location == top5_location[0])|(dataframe.location == top5_location[1])|(dataframe.location == top5_location[2])|(dataframe.location == top5_location[3]) | (dataframe.location == top5_location[4])]
new_df.head(3)

data_bangalore=new_df[['name','cuisines','location','rate', 'votes']]

C = data_bangalore['rate'].mean()
# print(C)


# Calculate the minimum number of votes required to be in the chart, m
m = data_bangalore['votes'].quantile(0.90)

# Filter out all qualified restaurants into a new DataFrame
qualified_restaurant = data_bangalore.copy().loc[data_bangalore['votes'] >= m]
# qualified_restaurant.shape

# Function that computes the weighted rating of each restaurant

# m = the minimum number of votes required to be in the chart
# C = Mean rate
def weighted_rating(x, m=m, C=C):
    v = x['votes']
    R = x['rate']
    # Calculation based on the weighted formula
    return (v/(v+m) * R) + (m/(m+v) * C)

# Define a new feature 'score' and calculate its value with `weighted_rating()`
qualified_restaurant['score'] = qualified_restaurant.apply(weighted_rating, axis=1)

#Sort movies based on score calculated above
qualified_restaurant = qualified_restaurant.sort_values('score', ascending=False)

#Print the top 15 res
print("Print the top 15 restaurants")
print(qualified_restaurant[['name','cuisines', 'location','votes', 'rate', 'score']].head(15))



 Model 1


Print the top 15 restaurants
                                name  \
Unnamed: 0                             
35270       Brooks and Bonds Brewery   
34684       Brooks and Bonds Brewery   
32624       Brooks and Bonds Brewery   
28400                   Dyu Art Cafe   
41531                     Jalsa Gold   
35234               Sultans of Spice   
26486               Sultans of Spice   
32041               Buff Buffet Buff   
8882                   Meghana Foods   
11456       Brooks and Bonds Brewery   
29615              Once Upon A Flame   
19995                   Mudpipe Cafe   
36925                         Stoner   
34697                   Crawl Street   
21293         Chulha Chauki Da Dhaba   

                                                     cuisines  \
Unnamed: 0                                                      
35270       Continental, Mediterranean, North Indian, Chin...   
34684       Continental, Mediterranean, North Indian, Chin...   
32624       Conti